# Modelo de OneClassSVM entrenado con 4F de datos Astro, y predicción sobre datos Unid (no identificados)

**Proyecto**: Detección de posibles fuentes de materia oscura usando ML en datos Fermi-LAT  
**Autor**: Marta Canino Romero  
**Fecha**: febrero-mayo 20225

---

## Descripción:

Este notebook aplica un modelo **One-Class SVM** entrenado con datos de fuentes astrofísicas conocidas (ASTRO) usando las siguientes características:

- E_peak
- beta
- sigma
- betaRel

Este modelo se entrena para identificar anomalías que puedan corresponder a posibles fuentes de materia oscura (UNIDs) en los datos no identificados del catálogo 4FGL.

---

## Objetivos específicos:

- Entrenar modelo OCSVM con 4 features
- Optimizar hiperparámetros (grid search sobre `nu` y `gamma`)
- Evaluar sobre datos de validación y prueba
- Aplicar modelo final sobre datos UNID para predicción
- Comparar los resultados de anomalía/outliers para OCSVM 4F con los resultados de probabilidad de DM de la ANN 4F (sobre UNIDs)

---

## Entrada de datos:

- '../data/astro_df.txt`

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib.patches import Rectangle

from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score, confusion_matrix, classification_report, ConfusionMatrixDisplay, roc_curve, auc
from sklearn.utils.multiclass import unique_labels

from mpl_toolkits.mplot3d import Axes3D

from itertools import combinations

from scipy import stats

import warnings
warnings.filterwarnings('ignore')

# Configurar estilo
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (15, 10)

In [ ]:
# data_path = "../data/astro_df.txt"

data_path = "../data/astro_data_with_labels.txt"

df_astro = pd.read_csv(data_path, sep='\s+')
print(f" Nombres de las columnas: {list(df_astro.columns)}")

# Renombramos la columna target por claridad
df_astro = df_astro.rename(columns={"astro_DM": "class"})

print(f"Dataset cargado. Forma: {df_astro.shape}")
print(f"Nombres de las columnas: {list(df_astro.columns)}")

display(df_astro.head())


In [ ]:

# Selección de características
features = ['Log(E_peak)', 'Log(beta)', 'Log(sigma)', 'Log(beta_Rel)']
"""Establecemos la columna objetivo aunque al ser un problema de detección de anomalías no la utilizaremos para entrenar el modelo.
Además de que todos los datos están etiquetados como 'astro_DM' = 0.0"""
target = 'class'

print(f"Modelo 4D con features: {features}")
print(f"Dimensionalidad del espacio: {len(features)}D")

# Comprobamos valores nulos
print("\n Valores faltantes por columna:")
print(df_astro[features + [target]].isnull().sum())

print("\n Muestra del dataset:")
display(df_astro[features + [target]].head())

In [ ]:
# Crear figura con todas las combinaciones 2D
features = ['Log(E_peak)', 'Log(beta)', 'Log(sigma)', 'Log(beta_Rel)']
feature_pairs = list(combinations(features, 2))

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for idx, (feat1, feat2) in enumerate(feature_pairs):
    ax = axes[idx]
    
    sns.scatterplot(
        data=df_astro,
        x=feat1,
        y=feat2,
        color="cornflowerblue",
        edgecolor='k',
        alpha=0.7,
        s=40,
        ax=ax
    )
    
    ax.set_title(f"2D ASTRO Data: {feat1} vs {feat2}")
    ax.set_xlabel(feat1)
    ax.set_ylabel(feat2)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Crear múltiples gráficos 3D con diferentes combinaciones
fig = plt.figure(figsize=(20, 6))

# Combinaciones 3D más representativas
combinations_3d = [
    ('Log(E_peak)', 'Log(beta)', 'Log(sigma)'),
    ('Log(E_peak)', 'Log(beta)', 'Log(beta_Rel)'),
    ('Log(E_peak)', 'Log(sigma)', 'Log(beta_Rel)'),
    ('Log(beta)', 'Log(sigma)', 'Log(beta_Rel)')
]

for idx, (feat1, feat2, feat3) in enumerate(combinations_3d[:3]):  # Solo 3 gráficos
    ax = fig.add_subplot(1, 3, idx+1, projection='3d')
    
    x = df_astro[feat1]
    y = df_astro[feat2] 
    z = df_astro[feat3]
    labels = df_astro['class']
    
    scatter = ax.scatter(x, y, z, c=labels, cmap='cool', edgecolor='k', alpha=0.7, s=30)
    
    ax.set_xlabel(feat1)
    ax.set_ylabel(feat2)
    ax.set_zlabel(feat3)
    ax.set_title(f'3D ASTRO Data: {feat1}, {feat2}, {feat3}')
    ax.view_init(elev=20, azim=30)

plt.tight_layout()
plt.show()

In [ ]:

# Seleccionamos las features dinámicamente
X = df_astro[features].values
y = df_astro[target].values

print(f"Forma del dataset: {X.shape}")
print(f"Distribución de clases: {np.unique(y, return_counts=True)}")

# Como todos los datos son clase 0, stratify no es necesario y puede causar errores
# Simplificamos a:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

print(f"Train: {X_train.shape[0]}, Val: {X_val.shape[0]}, Test: {X_test.shape[0]}")

In [ ]:

# Ejemplo de barrido en malla de hiperparámetros para OneClassSVM
# Sseleccionamos más abajo los valores manualmente para permitir al modelo obtener un nº de outliers razonable en las fuentes UNID
"""
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

nu_values    = [0.001, 0.002, 0.005, 0.01, 0.02, 0.05]
gamma_values = ['scale', 'auto', 0.002, 0.001, 0.01, 0.02, 0.1, 1, 10]

results        = []
best_outliers  = np.inf
best_model     = None
best_params    = {}

for nu in nu_values:
    for gamma in gamma_values:
        model = OneClassSVM(kernel='rbf', nu=nu, gamma=gamma)
        model.fit(X_train_scaled)  # ← usar X_train_scaled aquí

        # Predecir sobre validación escalada
        preds       = model.predict(X_val_scaled)       # 1=inlier, -1=outlier
        n_outliers  = np.sum(preds == -1)

        results.append({
            'nu': nu,
            'gamma': gamma,
            'val_outliers': n_outliers
        })

        # Elegir el modelo que arroje MENOS outliers
        if n_outliers < best_outliers:
            best_outliers = n_outliers
            best_model    = model
            best_params   = {'nu': nu, 'gamma': gamma}

# Mostrar resultados
print("Mejor combinación de hiperparámetros:")
print(f"   - nu    = {best_params['nu']}")
print(f"   - gamma = {best_params['gamma']}")
print(f"Outliers (val set): {best_outliers} de {len(X_val_scaled)} muestras")

df_results = pd.DataFrame(results)
display(df_results.sort_values(by='val_outliers'))
"""


In [ ]:
"""
PARAMETROS DE OneClassSVM:

- kernel: rbf (solo cambiar si conocemos la fórmula para la forma de la distribución)
- gamma: 
    - coeficiente del kernel 
    - usado para hiperplanos no lineares
    - influye en la forma de la frontera de decisión + el desempeño predictivo
    - define la influencia de un único ejemplo de entrenamiento
    - cuanto más grande, más cerca tienen que estar los otros ejemplos para 'verse afectados'
    - (por defecto 'scale' (antes 'auto')) 
- nu: 
    - límite superior de la fracción de errores permitidos en entrenamiento 
    - límite inferior de la fracción de vectores de soporte en relación con nº total de datos de entrenamiento
        -   ejemplo: si se establece en 0,05 se tiene la garantía de encontrar como máximo el 5 % de los datos de entrenamiento mal clasificados 
            y al menos el 5 % de los datos de entrenamiento siendo vectores de soporte
    - ( (0,1] - 0.5 por defecto )
"""

"""
EJEMPLOS DE OVERFITTING Y UNDERFITTING:

    Underfitting severo
        gamma = 0.001   # muy bajo
        nu = 0.5        # muy alto

    Overfitting severo
    gamma = 1.0     # muy alto
    nu = 0.0001     # extremadamente bajo
"""

"""
Aunque podríamos utilizar el barrido en malla de más arriba,
seleccionamos manualmente los valores de nu y gamma en base a experimentos anteriores 
para permitir al modelo obtener un nº de outliers razonable en las fuentes UNID que podamos analizar más adelante.
"""

# Parámetros optimizados para 4D
selected_gamma = 0.02  # 0.02 - Menor que 0.1 usado en 2D
selected_nu = 0.001    # 0.001 - Mantenemos la misma expectativa de outliers

print(f"Parámetros: γ={selected_gamma}, ν={selected_nu}")

# Entrenar modelo final con todos los datos astro (train + val)
X_final_train = np.vstack([X_train, X_val])
y_final_train = np.concatenate([y_train, y_val])

# Scaler final - SOLO ajustar con datos de entrenamiento
scaler_final = StandardScaler()
X_final_train_scaled = scaler_final.fit_transform(X_final_train)

In [ ]:

# Modelo final
final_model = OneClassSVM(kernel='rbf', gamma=selected_gamma, nu=selected_nu)
final_model.fit(X_final_train_scaled)

In [ ]:
#
#  Hacer predicciones con el modelo final entrenado sobre datos de prueba

# CRÍTICO: Solo transform (NO fit_transform) para datos de test
X_test_scaled = scaler_final.transform(X_test)

# Análisis de la distribución de scores
decision_scores_test = final_model.decision_function(X_test_scaled)

print(f"\nDecision scores estadísticas:")
print(f"  Media: {np.mean(decision_scores_test):.4f}")
print(f"  Std: {np.std(decision_scores_test):.4f}")
print(f"  Min: {np.min(decision_scores_test):.4f}")
print(f"  Max: {np.max(decision_scores_test):.4f}")

In [ ]:

# Evaluar en test
test_preds = final_model.predict(X_test_scaled)
test_labels = np.where(test_preds == 1, 0, 1)

# Separar inliers y outliers
inliers_mask = test_preds == 1
outliers_mask = test_preds == -1

n_inliers = np.sum(test_preds == 1)
n_outliers = np.sum(test_preds == -1)
total_samples = len(test_preds)

print(f"\nEvaluación en el conjunto de test:")
print(f"Total de muestras: {total_samples}")
print(f"Inliers detectados: {n_inliers} ({n_inliers/total_samples*100:.2f}%)")
print(f"Outliers detectados: {n_outliers} ({n_outliers/total_samples*100:.2f}%)")

In [ ]:
# Metricas
print("Matriz de confusión:")
cm = confusion_matrix(y_test, test_labels)
print(cm)

print(f"\nReporte de clasificación:")
print(classification_report(y_test, test_labels, target_names=unique_labels(y_test, test_labels).astype(str)))

In [ ]:
# Análisis de outliers detectados
if n_outliers > 0:
    outlier_features = X_test[outliers_mask]
    print(f"\nCaracterísticas de los {n_outliers} outliers detectados:")
    for i, feature in enumerate(features):
        values = outlier_features[:, i]
        print(f"  {feature}: [{np.min(values):.3f}, {np.max(values):.3f}]")
        
    # Decision scores de los outliers
    outlier_scores = decision_scores_test[outliers_mask]
    print(f"\nDecision scores de outliers:")
    print(f"  Media: {np.mean(outlier_scores):.4f}")
    print(f"  Rango: [{np.min(outlier_scores):.4f}, {np.max(outlier_scores):.4f}]")

In [ ]:
feature_names = ['Log(E_peak)', 'Log(beta)', 'Log(sigma)', 'Log(beta_Rel)']

feature_pairs = list(combinations(range(len(feature_names)), 2))
n_pairs = len(feature_pairs)

# Grid layout
cols = 3
rows = int(np.ceil(n_pairs / cols))
fig, axes = plt.subplots(rows, cols, figsize=(18, 6*rows))

# Asegurar que axes sea siempre 2D
if rows == 1:
    axes = axes.reshape(1, -1)
if cols == 1:
    axes = axes.reshape(-1, 1)

# Calcular máscaras de inliers/outliers una sola vez
predictions = final_model.predict(X_test_scaled)
inliers_mask = predictions == 1
outliers_mask = predictions == -1

# Crear visualizaciones 2D
for i, (feature_idx1, feature_idx2) in enumerate(feature_pairs):
    row = i // cols
    col = i % cols
    ax = axes[row, col]
    
    # Dimensiones que no se están mostrando
    other_dims = [j for j in range(len(feature_names)) if j not in [feature_idx1, feature_idx2]]
    
    # Usar medianas en lugar de medias para las dimensiones no mostradas
    median_values = [np.median(X_test_scaled[:, dim]) for dim in other_dims]
    
    # Límites para el grid
    x_min = X_test_scaled[:, feature_idx1].min() - 0.5
    x_max = X_test_scaled[:, feature_idx1].max() + 0.5
    y_min = X_test_scaled[:, feature_idx2].min() - 0.5
    y_max = X_test_scaled[:, feature_idx2].max() + 0.5
    
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 150), 
                         np.linspace(y_min, y_max, 150))
    
    # Crear grid 4D completo de manera más eficiente
    grid_size = xx.ravel().shape[0]
    grid_4d = np.zeros((grid_size, len(feature_names)))
    
    # Asignar valores para las dos dimensiones que se muestran
    grid_4d[:, feature_idx1] = xx.ravel()
    grid_4d[:, feature_idx2] = yy.ravel()
    
    # Asignar medianas para las otras dimensiones
    for j, other_dim in enumerate(other_dims):
        grid_4d[:, other_dim] = median_values[j]
    
    # Calcular función de decisión
    Z = final_model.decision_function(grid_4d)
    Z = Z.reshape(xx.shape)
    
    # Asegurar que tenemos un rango válido para los niveles
    z_min = Z.min()
    z_max = 0  # OneClassSVM usa 0 como frontera
    
    if z_min >= z_max:
        z_max = z_min + 1e-3  # Añadir pequeño valor para evitar error
    
    # Crear niveles para contourf
    levels = np.linspace(z_min, z_max, 8)
    
    # Plotear contornos
    contour_fill = ax.contourf(xx, yy, Z, levels=levels, cmap='RdYlBu_r', alpha=0.6)
    ax.contour(xx, yy, Z, levels=[0], linewidths=2, colors='red', linestyles='--')
    
    # Plotear puntos
    ax.scatter(X_test_scaled[inliers_mask, feature_idx1], 
              X_test_scaled[inliers_mask, feature_idx2], 
              c='darkblue', s=20, alpha=0.8, 
              label=f'Inliers ({np.sum(inliers_mask)})')
    
    if np.any(outliers_mask):
        ax.scatter(X_test_scaled[outliers_mask, feature_idx1], 
                  X_test_scaled[outliers_mask, feature_idx2], 
                  c='red', s=50, alpha=1.0, marker='D', 
                  label=f'Outliers ({np.sum(outliers_mask)})')
    
    # Etiquetas y título
    ax.set_xlabel(feature_names[feature_idx1])
    ax.set_ylabel(feature_names[feature_idx2])
    
    # Obtener los parámetros del modelo
    gamma_val = final_model.gamma if isinstance(final_model.gamma, (int, float)) else final_model.gamma
    nu_val = final_model.nu
    
    ax.set_title(f'{feature_names[feature_idx1]} vs {feature_names[feature_idx2]}\n'
                f'ν={nu_val}, γ={gamma_val}\n'
                f'Outliers: {np.sum(outliers_mask)}/{len(X_test_scaled)}')
    
    ax.grid(True, alpha=0.3)
    ax.legend()

# Ocultar subplots vacíos si los hay
for i in range(len(feature_pairs), rows * cols):
    row = i // cols
    col = i % cols
    if row < axes.shape[0] and col < axes.shape[1]:
        axes[row, col].set_visible(False)

plt.tight_layout()
plt.show()

# Información adicional del modelo
print(f"\nModelo One-Class SVM 4F:")
print(f"  - Kernel: {final_model.kernel}")
print(f"  - Gamma: {final_model.gamma}")
print(f"  - Nu: {final_model.nu}")
print(f"  - Número de vectores soporte: {final_model.n_support_[0]}")
print(f"  - Outliers detectados: {np.sum(outliers_mask)}/{len(X_test_scaled)} ({100*np.sum(outliers_mask)/len(X_test_scaled):.2f}%)")

In [ ]:
feature_triplets = list(combinations(range(len(feature_names)), 3))
n_triplets = len(feature_triplets)

# Grid layout 3D
cols_3d = 2
rows_3d = int(np.ceil(n_triplets / cols_3d))
fig = plt.figure(figsize=(15, 7*rows_3d))

# Crear visualizaciones 3D
for i, (f1_idx, f2_idx, f3_idx) in enumerate(feature_triplets):
    ax = fig.add_subplot(rows_3d, cols_3d, i+1, projection='3d')
    
    # Plotear inliers con borde para más definición
    ax.scatter(X_test_scaled[inliers_mask, f1_idx], 
               X_test_scaled[inliers_mask, f2_idx], 
               X_test_scaled[inliers_mask, f3_idx],
               c='lightblue', s=30, alpha=0.7, 
               edgecolor='blue', linewidth=0.5,
               label=f'Inliers ({np.sum(inliers_mask)})')
    
    if np.any(outliers_mask):
        ax.scatter(X_test_scaled[outliers_mask, f1_idx], 
                   X_test_scaled[outliers_mask, f2_idx], 
                   X_test_scaled[outliers_mask, f3_idx],
                   c='red', s=100, alpha=1.0, marker='D', 
                   edgecolor='darkred', linewidth=1,
                   label=f'Outliers ({np.sum(outliers_mask)})')
    
    ax.set_xlabel(feature_names[f1_idx])
    ax.set_ylabel(feature_names[f2_idx])
    ax.set_zlabel(feature_names[f3_idx])
    ax.set_title(f'{feature_names[f1_idx]} vs {feature_names[f2_idx]} vs {feature_names[f3_idx]}\n'
                f'ν={selected_nu}, γ={selected_gamma} | Outliers: {np.sum(outliers_mask)}/{len(X_final_train)}')
    ax.legend()

plt.tight_layout()
plt.show()


# Importar UNIDs

In [ ]:

# Cargar datos UnIDs

# unids_path = "../data/unids_log.txt"
unids_path = "../data/unids_transformed_complete.txt"

# df_unids = pd.read_csv(unids_path, sep='\s+')
df_unids = pd.read_csv(unids_path, sep='\t')

print("Datos UnIDs cargados:")
print(f"Shape: {df_unids.shape}")
print("Primeras filas:")
display(df_unids.head())

# Verificar columnas disponibles
print(f"Columnas disponibles: {list(df_unids.columns)}")

In [ ]:

# Extraer características 4F
feature_cols = ["Log(E_peak)", "Log(beta)", "Log(sigma)", "Log(beta_Rel)"]
X_unids = df_unids[feature_cols].values

print(f"Características extraídas: {feature_cols}")
print(f"Shape características UnIDs: {X_unids.shape}")

# Predecir sobre UNIDs

In [ ]:

# Escalar características UnIDs
X_unids_scaled = scaler_final.transform(X_unids)

decision_scores = final_model.decision_function(X_unids_scaled)

unids_preds = final_model.predict(X_unids_scaled)

inliers_mask = unids_preds == 1
outliers_mask = unids_preds == -1

total_unids = len(unids_preds)
n_outliers = np.sum(unids_preds == -1)
n_inliers = np.sum(unids_preds == 1)
outlier_percentage = n_outliers / total_unids * 100
inlier_percentage = n_inliers / total_unids * 100

# Máscaras para análisis posterior
inliers_mask = unids_preds == 1
outliers_mask = unids_preds == -1

print(f"RESULTADOS DE PREDICCIÓN:")
print(f"  - Total UnIDs analizadas: {len(unids_preds)}")
print(f"  - Outliers detectados: {n_outliers} ({outlier_percentage:.1f}%)")
print(f"  - Inliers detectados: {n_inliers} ({inlier_percentage:.1f}%)")

print(f"Outliers detectados con sus IDs:")

outlier_ids = df_unids.loc[outliers_mask, 'number']
print(outlier_ids)

In [ ]:

# Calcular anomaly scores y rankings
# Invertir scores: valores más altos = más anómalos
anom_scores = -decision_scores

# Escalar a percentiles [0, 100]
anom_percent = MinMaxScaler(feature_range=(0, 100)).fit_transform(anom_scores.reshape(-1, 1)).flatten()

# Agregar resultados al DataFrame
df_unids_results = df_unids.copy()
df_unids_results["svm_score"] = decision_scores
df_unids_results["prediction"] = unids_preds
df_unids_results["Anomaly_Score"] = anom_scores
df_unids_results["Anomaly_Rank(%)"] = anom_percent

In [ ]:

# Filtrar solo outliers y ordenar por anomaly rank
outliers_only = df_unids_results[df_unids_results["prediction"] == -1].copy()
outliers_sorted = outliers_only.sort_values(by="Anomaly_Rank(%)", ascending=False)

# Top 10 outliers más anómalos
top_10_outliers = outliers_sorted.head(10)
top_10_indices = top_10_outliers.index

In [ ]:

print(f"TOP 10 CANDIDATOS MÁS ANÓMALOS:")
# Mostrar información relevante
display_cols = ['Log(E_peak)', 'Log(beta)', 'Log(sigma)', 'Log(beta_Rel)', 
                'svm_score', 'Anomaly_Score', 'Anomaly_Rank(%)']

if 'number' in df_unids_results.columns:
    display_cols = ['number'] + display_cols

display(top_10_outliers[display_cols].round(4))

In [ ]:
"""
# Comprobamos los valores mínimos y máximos de astro_df
print("\nValores mínimos y máximos de las características en astro_df:")
for feature in features:
    min_val = df_astro[feature].min()
    max_val = df_astro[feature].max()
    print(f"{feature}: [{min_val:.4f}, {max_val:.4f}]")
    """
# Comprobamos los valores mínimos y máximos de unids
print("\nValores mínimos y máximos de las características en unids:")
for feature in feature_cols:
    min_val = df_unids[feature].min()
    max_val = df_unids[feature].max()
    print(f"{feature}: [{min_val:.4f}, {max_val:.4f}]")

In [ ]:
print(f"\nTOP 10 ANOMALÍAS ETIQUETADAS:")
for i, idx in enumerate(top_10_indices):
    if 'number' in df_unids_results.columns:
        unid_id = int(df_unids_results.loc[idx, 'number'])
        rank = df_unids_results.loc[idx, 'Anomaly_Rank(%)']
        print(f"  {i+1:2d}. UNID {unid_id:3d} (Rank: {rank:5.1f}%)")
    else:
        rank = df_unids_results.loc[idx, 'Anomaly_Rank(%)']
        print(f"  {i+1:2d}. Índice {idx:3d} (Rank: {rank:5.1f}%)")

In [ ]:

# Estadísticas de los outliers
print(f"\nESTADÍSTICAS DE OUTLIERS:")
print(f"  - Anomaly Score promedio: {outliers_only['Anomaly_Score'].mean():.4f} ± {outliers_only['Anomaly_Score'].std():.4f}")
print(f"  - Anomaly Rank promedio: {outliers_only['Anomaly_Rank(%)'].mean():.1f}% ± {outliers_only['Anomaly_Rank(%)'].std():.1f}%")
print(f"  - SVM Score promedio: {outliers_only['svm_score'].mean():.4f} ± {outliers_only['svm_score'].std():.4f}")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Análisis Estadístico del Modelo OCSVM 4F en UNIDs', 
             fontsize=16, fontweight='bold', y=0.98)

# 1. Distribución de Decision Scores CON ESTADÍSTICAS
ax1 = axes[0, 0]
n, bins, patches = ax1.hist(decision_scores, bins=50, alpha=0.7, color='skyblue', 
                           edgecolor='black', density=True)

# Añadir curva de densidad suavizada
x_smooth = np.linspace(decision_scores.min(), decision_scores.max(), 100)
density = stats.gaussian_kde(decision_scores)
ax1.plot(x_smooth, density(x_smooth), 'navy', linewidth=2, label='Densidad estimada')

# Líneas de referencia
ax1.axvline(0, color='red', linestyle='--', linewidth=2, label='Frontera (score=0)')
ax1.axvline(decision_scores.mean(), color='green', linestyle='--', linewidth=2,
           label=f'Media: {decision_scores.mean():.3f}')
ax1.axvline(np.median(decision_scores), color='orange', linestyle='--', linewidth=2,
           label=f'Mediana: {np.median(decision_scores):.3f}')

# Estadísticas adicionales
std_dev = decision_scores.std()
ax1.fill_betweenx([0, ax1.get_ylim()[1]], decision_scores.mean()-std_dev, 
                  decision_scores.mean()+std_dev, alpha=0.2, color='green', 
                  label=f'±1σ: {std_dev:.3f}')

ax1.set_xlabel('SVM Decision Score')
ax1.set_ylabel('Densidad de Probabilidad')
ax1.set_title('Distribución de Decision Scores\n(Análisis de Normalidad)')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)

# Texto con estadísticas
stats_text = f'n = {len(decision_scores)}\nSkewness: {stats.skew(decision_scores):.3f}\nKurtosis: {stats.kurtosis(decision_scores):.3f}'
ax1.text(0.02, 0.98, stats_text, transform=ax1.transAxes, fontsize=9, 
         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# 2. Distribución de Anomaly Ranks CON PERCENTILES
ax2 = axes[0, 1]
ax2.hist(anom_percent, bins=50, alpha=0.7, color='lightcoral', edgecolor='black')

# Percentiles importantes
percentiles = [50, 75, 90, 95, 99]
colors = ['blue', 'orange', 'red', 'darkred', 'purple']
for p, color in zip(percentiles, colors):
    value = np.percentile(anom_percent, p)
    ax2.axvline(value, color=color, linestyle='--', linewidth=2, 
               label=f'P{p}: {value:.1f}%')

ax2.set_xlabel('Anomaly Rank (%)')
ax2.set_ylabel('Frecuencia')
ax2.set_title('Distribución de Anomaly Rankings\n(Análisis de Percentiles)')
ax2.legend(fontsize=9, loc='upper left')
ax2.grid(True, alpha=0.3)

# 3. Comparación Inliers vs Outliers
ax3 = axes[1, 0]
outliers_scores = anom_percent[unids_preds == -1]
inliers_scores = anom_percent[unids_preds == 1]

# Histograma
bins = np.linspace(0, 100, 31)
ax3.hist([inliers_scores, outliers_scores], bins=bins, alpha=0.7,
         color=['lightblue', 'red'], label=[f'Inliers (n={len(inliers_scores)})', 
                                           f'Outliers (n={len(outliers_scores)})'],
         edgecolor='black')

# Añadir líneas de medias
ax3.axvline(inliers_scores.mean(), color='blue', linestyle='-', linewidth=2,
           label=f'Media Inliers: {inliers_scores.mean():.1f}%')
ax3.axvline(outliers_scores.mean(), color='darkred', linestyle='-', linewidth=2,
           label=f'Media Outliers: {outliers_scores.mean():.1f}%')

ax3.set_xlabel('Anomaly Rank (%)')
ax3.set_ylabel('Frecuencia')
ax3.set_title('Separación Inliers vs Outliers\n(Validación del Modelo)')
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3)

# Test estadístico
stat, p_value = stats.mannwhitneyu(outliers_scores, inliers_scores, alternative='greater')
test_text = f'Mann-Whitney U test:\np-value: {p_value:.2e}'
ax3.text(0.02, 0.98, test_text, transform=ax3.transAxes, fontsize=9,
         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# 4. Scatter con regiones de interés
ax4 = axes[1, 1]
colors = ['red' if pred == -1 else 'blue' for pred in unids_preds]
sizes = [40 if pred == -1 else 20 for pred in unids_preds]

scatter = ax4.scatter(decision_scores, anom_percent, c=colors, alpha=0.6, s=sizes,
                     edgecolors='black', linewidth=0.5)

# Regiones de interés
ax4.axvline(0, color='black', linestyle='--', alpha=0.7, linewidth=2, label='Frontera decisión')
ax4.axhline(90, color='red', linestyle='--', alpha=0.7, label='Top 10% anomalías')
ax4.axhline(95, color='darkred', linestyle='--', alpha=0.7, label='Top 5% anomalías')

# Destacar zona de interés (outliers extremos)
extreme_mask = (unids_preds == -1) & (anom_percent > 95)
if np.any(extreme_mask):
    ax4.scatter(decision_scores[extreme_mask], anom_percent[extreme_mask], 
               s=100, facecolors='none', edgecolors='yellow', linewidth=3,
               label=f'Outliers extremos (n={np.sum(extreme_mask)})')

# Correlación
correlation = np.corrcoef(decision_scores, anom_percent)[0, 1]
ax4.text(0.02, 0.98, f'Correlación: {correlation:.3f}', transform=ax4.transAxes, 
         fontsize=11, verticalalignment='top', fontweight='bold',
         bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))

ax4.set_xlabel('SVM Decision Score')
ax4.set_ylabel('Anomaly Rank (%)')
ax4.set_title('Relación Decision Score vs Anomaly Rank\n(Coherencia del Modelo)')
ax4.legend(fontsize=9, loc='lower right')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# GRID DE VISUALIZACIONES 2D DE UNIDs CON IDs DE ANOMALÍAS
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle(f'Modelo OCSVM 4F: Proyecciones 2D de UNIDs\n'
            f'Outliers detectados: {np.sum(outliers_mask)} de {len(unids_preds)}', 
            fontsize=14, fontweight='bold')

# Todas las combinaciones 2D de las 4 características
feature_pairs = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
feature_names = ['Log(E_peak)', 'Log(beta)', 'Log(sigma)', 'Log(beta_Rel)']

# Separar inliers y outliers
inliers_mask = unids_preds == 1
outliers_mask = unids_preds == -1
top_10_indices = top_10_outliers.index

for idx, (i, j) in enumerate(feature_pairs):
    ax = axes[idx//3, idx%3]
    
    # Plot inliers con el estilo gold
    ax.scatter(X_unids[inliers_mask, i], X_unids[inliers_mask, j],
               c='gold', alpha=0.6, s=30, edgecolors='black', linewidth=0.3,
               label=f'Inliers (astro-like) ({np.sum(inliers_mask)})')
    
    # Plot outliers con gradient de colores según anomaly rank
    if np.any(outliers_mask):
        scatter = ax.scatter(X_unids[outliers_mask, i], X_unids[outliers_mask, j],
                            c=anom_percent[outliers_mask], cmap='Reds', s=60,
                            alpha=0.8, edgecolors='black', linewidth=0.5, 
                            label=f'Outliers (anomalous) ({np.sum(outliers_mask)})')
        
        """# Más control sobre el colorbar
        if idx == 2:  # subplot (0,2)
            cbar = plt.colorbar(scatter, ax=ax)
            cbar.set_label('Anomaly Rank %', fontsize=10)
            cbar.ax.tick_params(labelsize=9)"""
    
    # Destacar top 10 con círculos amarillos
    if len(top_10_indices) > 0:
        ax.scatter(X_unids[top_10_indices, i], X_unids[top_10_indices, j],
                   s=120, facecolors='none', edgecolors='yellow', linewidth=3,
                   label=f'Top 10 Anomalies', marker='o')
        
        # Añadir etiquetas de ID para top 10
        for unid_idx in top_10_indices:
            x = X_unids[unid_idx, i]
            y = X_unids[unid_idx, j]
            
            # Obtener el número del UNID
            if 'number' in df_unids_results.columns:
                unid_id = int(df_unids_results.loc[unid_idx, 'number'])
            else:
                unid_id = unid_idx
            
            # Posicionar texto ligeramente desplazado para evitar solapamiento
            offset_x = (X_unids[:, i].max() - X_unids[:, i].min()) * 0.02
            offset_y = (X_unids[:, j].max() - X_unids[:, j].min()) * 0.02
            
            ax.text(x + offset_x, y + offset_y, str(unid_id), 
                   color='black', fontsize=8, fontweight='bold',
                   ha='left', va='bottom',
                   bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.7))
    
    # Configuración de ejes y título
    ax.set_xlabel(feature_names[i])
    ax.set_ylabel(feature_names[j])
    ax.set_title(f'{feature_names[i]} vs {feature_names[j]}\n'
            f'UNIDs: Modelo 4F - Top 10 Anomalías Etiquetadas')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

# Sin colorbar, pero con información en la leyenda:
if np.any(outliers_mask):
    scatter = ax.scatter(X_unids[outliers_mask, i], X_unids[outliers_mask, j],
                        c=anom_percent[outliers_mask], cmap='Reds', s=60,
                        alpha=0.8, edgecolors='black', linewidth=0.5, 
                        label=f'Outliers (color = Anomaly Rank)')
    
    # Añadir texto explicativo
    ax.text(0.02, 0.98, 'Color: Rojo más intenso = Mayor anomalía', 
           transform=ax.transAxes, fontsize=8, 
           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

In [ ]:
# GRID DE VISUALIZACIONES 3D DE UNIDs
feature_names = ['Log(E_peak)', 'Log(beta)', 'Log(sigma)', 'Log(beta_Rel)']
feature_triplets = list(combinations(range(len(feature_names)), 3))
n_triplets = len(feature_triplets)

# Grid layout 3D optimizado
cols_3d = 2
rows_3d = int(np.ceil(n_triplets / cols_3d))
fig = plt.figure(figsize=(15, 7*rows_3d))

"""# Separar inliers y outliers
inliers_mask = unids_preds == 1
outliers_mask = unids_preds == -1
top_10_indices = top_10_outliers.index

# Conteos para leyenda unificada
n_inliers = np.sum(inliers_mask)
n_outliers = np.sum(outliers_mask)
"""

n_top10 = len(top_10_indices)

# Crear visualizaciones 3D
for i, (f1_idx, f2_idx, f3_idx) in enumerate(feature_triplets):
    ax = fig.add_subplot(rows_3d, cols_3d, i+1, projection='3d')
    
    # Plot inliers
    ax.scatter(X_unids[inliers_mask, f1_idx], 
               X_unids[inliers_mask, f2_idx], 
               X_unids[inliers_mask, f3_idx],
               c='gold', s=25, alpha=0.6, 
               edgecolor='black', linewidth=0.3)
    
    # Plot outliers con gradient de color según anomaly rank
    if np.any(outliers_mask):
        scatter = ax.scatter(X_unids[outliers_mask, f1_idx], 
                           X_unids[outliers_mask, f2_idx], 
                           X_unids[outliers_mask, f3_idx],
                           c=anom_percent[outliers_mask], cmap='Reds', s=60,
                           alpha=0.8, edgecolors='black', linewidth=0.4)
    
    # Destacar top 10 anomalías
    if len(top_10_indices) > 0:
        ax.scatter(X_unids[top_10_indices, f1_idx], 
                   X_unids[top_10_indices, f2_idx], 
                   X_unids[top_10_indices, f3_idx],
                   s=120, facecolors='none', edgecolors='yellow', 
                   linewidth=2.5)
        
        # Añadir etiquetas de ID con mejor posicionamiento
        for idx in top_10_indices:
            x = X_unids[idx, f1_idx]
            y = X_unids[idx, f2_idx] 
            z = X_unids[idx, f3_idx]
            
            # Obtener el número del UNID
            if 'number' in df_unids_results.columns:
                unid_id = int(df_unids_results.loc[idx, 'number'])
            else:
                unid_id = idx
            
            ax.text(x, y, z, str(unid_id), fontsize=7, color='black', 
                   fontweight='bold', ha='center', va='bottom',
                   bbox=dict(boxstyle='round,pad=0.15', facecolor='white', 
                            alpha=0.8, edgecolor='none'))
    
    # Configuración de ejes más compacta
    ax.set_xlabel(feature_names[f1_idx], fontsize=9)
    ax.set_ylabel(feature_names[f2_idx], fontsize=9)
    ax.set_zlabel(feature_names[f3_idx], fontsize=9)
    
    # Título más compacto
    title_short = f'{feature_names[f1_idx]} vs {feature_names[f2_idx]} vs {feature_names[f3_idx]}'
    ax.set_title(title_short, fontsize=10, pad=10)
    
    # Ajustar tamaño de ticks
    ax.tick_params(axis='both', which='major', labelsize=8)

"""
# Colorbar unificado en la parte superior
if np.any(outliers_mask):
    # Crear un scatter dummy para el colorbar
    dummy_scatter = plt.scatter([], [], c=[], cmap='Reds', s=60, alpha=0.8)
    dummy_scatter.set_array(anom_percent[outliers_mask])
    
    # Colorbar en la parte superior
    cbar_ax = fig.add_axes([0.15, 0.95, 0.7, 0.02])  # [left, bottom, width, height]
    cbar = plt.colorbar(dummy_scatter, cax=cbar_ax, orientation='horizontal')
    cbar.set_label('Anomaly Rank %', fontsize=11, labelpad=10)
"""
# Leyenda unificada en la parte inferior
legend_elements = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='gold', 
               markeredgecolor='black', markersize=8, alpha=0.8,
               label=f'Inliers (astro-like): {n_inliers}'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', 
               markeredgecolor='black', markersize=8, alpha=0.8,
               label=f'Outliers (anomalous): {n_outliers}'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='none', 
               markeredgecolor='yellow', markersize=10, markeredgewidth=2,
               label=f'Top 10 Anomalies: {n_top10}')
]

fig.legend(handles=legend_elements, loc='lower center', ncol=3, 
          bbox_to_anchor=(0.5, 0.02), fontsize=11, frameon=True, 
          fancybox=True, shadow=True)

# Ajustar layout con más espacio para leyenda y colorbar
plt.subplots_adjust(top=0.90, bottom=0.15, left=0.05, right=0.95, 
                   hspace=0.3, wspace=0.2)

fig.suptitle('Modelo OCSVM 4F: Proyecciones 3D de UNIDs\n'
            f'Outliers detectados: {n_outliers} de {len(unids_preds)}', 
            fontsize=14, fontweight='bold', y=0.97)
plt.show()


In [ ]:
# GRID DE VISUALIZACIONES 3D DE UNIDs - SIN COLORBAR
feature_names = ['Log(E_peak)', 'Log(beta)', 'Log(sigma)', 'Log(beta_Rel)']
feature_triplets = list(combinations(range(len(feature_names)), 3))

# Layout compacto 2x2
fig = plt.figure(figsize=(12, 10))

"""# Separar inliers y outliers
inliers_mask = unids_preds == 1
outliers_mask = unids_preds == -1
top_10_indices = top_10_outliers.index

n_inliers = np.sum(inliers_mask)
n_outliers = np.sum(outliers_mask)"""

# Crear visualizaciones 3D
for i, (f1_idx, f2_idx, f3_idx) in enumerate(feature_triplets):
    ax = fig.add_subplot(2, 2, i+1, projection='3d')
    
    # Plot inliers
    ax.scatter(X_unids[inliers_mask, f1_idx], 
               X_unids[inliers_mask, f2_idx], 
               X_unids[inliers_mask, f3_idx],
               c='lightblue', s=20, alpha=0.6, 
               edgecolor='blue', linewidth=0.2,
               label=f'Inliers ({n_inliers})')
    
    # Plot outliers en rojo sólido
    if np.any(outliers_mask):
        ax.scatter(X_unids[outliers_mask, f1_idx], 
                   X_unids[outliers_mask, f2_idx], 
                   X_unids[outliers_mask, f3_idx],
                   c='red', s=60, alpha=0.9, 
                   edgecolors='darkred', linewidth=0.4,
                   label=f'Outliers ({n_outliers})')
    
    # Top 10 con círculos amarillos
    if len(top_10_indices) > 0:
        ax.scatter(X_unids[top_10_indices, f1_idx], 
                   X_unids[top_10_indices, f2_idx], 
                   X_unids[top_10_indices, f3_idx],
                   s=100, facecolors='none', edgecolors='yellow', 
                   linewidth=3, label='Top 10')
    
    # Configuración
    ax.set_xlabel(feature_names[f1_idx], fontsize=10)
    ax.set_ylabel(feature_names[f2_idx], fontsize=10)
    ax.set_zlabel(feature_names[f3_idx], fontsize=10)
    
    # Título corto
    ax.set_title(f'{feature_names[f1_idx][:8]} vs {feature_names[f2_idx][:8]} vs {feature_names[f3_idx][:8]}', 
                fontsize=11)
    
    # Leyenda solo en el primer subplot
    if i == 0:
        ax.legend(fontsize=9, loc='upper left')
    
    ax.tick_params(labelsize=8)

# Título general con información clave
fig.suptitle(f'Modelo OCSVM 4F: Proyecciones 3D\n'
            f'{n_outliers} Outliers detectados de {len(unids_preds)} UNIDs', 
            fontsize=13, fontweight='bold')

# Layout más compacto
plt.tight_layout()
plt.subplots_adjust(top=0.90)
plt.show()

# Análisis de consenso entre modelos OCSVM 2F y OCSVM 4F - Resultados de UNIDs

In [ ]:
# Análisis de intersección entre modelos
candidatos_2f = set([1054, 275, 1116, 1017])  # Top 4 del modelo 2F
candidatos_4f = set([307, 285, 166, 923, 1116])  # Top 5 del modelo 4F

consenso = candidatos_2f.intersection(candidatos_4f)
unicos_2f = candidatos_2f - candidatos_4f
unicos_4f = candidatos_4f - candidatos_2f

print(f"\nAnálisis de intersección entre modelos:")
print(f"  - Candidatos comunes (consenso): {sorted(consenso)}")
print(f"  - Candidatos únicos en 2F: {sorted(unicos_2f)}")
print(f"  - Candidatos únicos en 4F: {sorted(unicos_4f)}")


## Guardar modelo para API extra

In [ ]:
# Export trained model to disk
import joblib
model_path = "../models/model_ocsvm_4f.pkl"
joblib.dump(final_model, model_path)
print(f"Modelo One-Class SVM guardado en: {model_path}")

# OCSVM 4F vs ANN 4F - Comparar los resultados (sobre UNIDs) con los de la red neuronal

In [ ]:
print("ANÁLISIS COMPARATIVO: ANN 4F vs OCSVM 4F\n")

# Cargar resultados de consenso ANN 4F
ann_consensus_path = "../data/results/ann/4F/consensus_analysis_ann_4f_improved.csv"
df_ann_consensus = pd.read_csv(ann_consensus_path)

print(f"Datos ANN 4F cargados:")
print(f"  - Shape: {df_ann_consensus.shape}")
print(f"  - Columnas principales: Source_ID, Mean_Consensus, Std_Consensus\n")

df_ann_consensus.head()

In [ ]:
# Obtener candidatos OCSVM 2F (outliers detectados)
ocsvm_outliers = set()
if np.any(unids_preds == -1):
    outlier_indices = np.where(unids_preds == -1)[0]
    for idx in outlier_indices:
        if 'number' in df_unids_results.columns:
            unid_id = int(df_unids_results.loc[idx, 'number'])
            ocsvm_outliers.add(unid_id)

# Top 10 OCSVM por anomaly rank
ocsvm_top10 = set()
if len(df_unids_results) > 0:
    top_10_ocsvm = df_unids_results.nlargest(10, 'Anomaly_Rank(%)')
    for idx, row in top_10_ocsvm.iterrows():
        if 'number' in row:
            ocsvm_top10.add(int(row['number']))

print(f"Candidatos OCSVM 4F:")
print(f"  - Outliers detectados: {len(ocsvm_outliers)} → {sorted(ocsvm_outliers)}")
print(f"  - Top 10 anomalías: {len(ocsvm_top10)} → {sorted(ocsvm_top10)}\n")

In [ ]:
# Obtener candidatos ANN 2F
ann_top4 = set(df_ann_consensus.head(4)['Source_ID'].astype(int))
ann_top10 = set(df_ann_consensus.head(10)['Source_ID'].astype(int))

# Candidatos ANN por threshold de consenso
consensus_threshold = 0.7  # Definir umbral de consenso
high_consensus_mask = df_ann_consensus['Mean_Prob_Consensus'] >= consensus_threshold
ann_high_consensus = set()
if high_consensus_mask.any():
    ann_high_consensus = set(df_ann_consensus[high_consensus_mask]['Source_ID'].astype(int))

print(f"Candidatos ANN 4F:")
print(f"  - Top 4: {sorted(ann_top4)}")
print(f"  - Top 10: {sorted(ann_top10)}")
print(f"  - Alto consenso (≥{consensus_threshold}): {len(ann_high_consensus)} → {sorted(ann_high_consensus)}\n")


In [ ]:
# VERIFICACIÓN DE CONSISTENCIA DE DATOS
print("VERIFICACIÓN DE CONSISTENCIA DE DATOS\n")

# Obtener algunos candidatos para verificar
verification_candidates = list(ocsvm_outliers.union(ann_top4))[:5]  # Primeros 5 para verificar

print("Verificando que Source_ID (ANN) = number (OCSVM) corresponden a las mismas fuentes:\n")

consistent_data = True
for candidate in verification_candidates:
    # Buscar en datos ANN
    ann_row = df_ann_consensus[df_ann_consensus['Source_ID'] == candidate]
    # Buscar en datos OCSVM
    ocsvm_row = df_unids_results[df_unids_results['number'] == candidate]
    
    if len(ann_row) > 0 and len(ocsvm_row) > 0:
        # Extraer valores de features
        ann_epeak = ann_row['log_E_peak'].iloc[0] if 'log_E_peak' in ann_row.columns else None
        ann_beta = ann_row['log_Beta'].iloc[0] if 'log_Beta' in ann_row.columns else None
        
        ocsvm_epeak = ocsvm_row['Log(E_peak)'].iloc[0] if 'Log(E_peak)' in ocsvm_row.columns else None
        ocsvm_beta = ocsvm_row['Log(beta)'].iloc[0] if 'Log(beta)' in ocsvm_row.columns else None
        
        if ann_epeak is not None and ocsvm_epeak is not None:
            epeak_diff = abs(ann_epeak - ocsvm_epeak)
            beta_diff = abs(ann_beta - ocsvm_beta)
            
            # Tolerancia para diferencias de precisión numérica
            tolerance = 1e-6
            epeak_match = epeak_diff < tolerance
            beta_match = beta_diff < tolerance
            
            print(f"UNID {candidate}:")
            print(f"  Log(E_peak): ANN={ann_epeak:.6f}, OCSVM={ocsvm_epeak:.6f}, diff={epeak_diff:.2e} {'✓' if epeak_match else '✗'}")
            print(f"  Log(beta):   ANN={ann_beta:.6f}, OCSVM={ocsvm_beta:.6f}, diff={beta_diff:.2e} {'✓' if beta_match else '✗'}")
            
            if not (epeak_match and beta_match):
                consistent_data = False
                print(f"  ⚠ INCONSISTENCIA detectada en UNID {candidate}")
            print()
    else:
        print(f"UNID {candidate}: No encontrado en uno de los datasets")
        if len(ann_row) == 0:
            print(f"  - Falta en datos ANN")
        if len(ocsvm_row) == 0:
            print(f"  - Falta en datos OCSVM")
        print()

if consistent_data:
    print("✓ VERIFICACIÓN EXITOSA: Los datos son consistentes entre ANN y OCSVM")
else:
    print("✗ ADVERTENCIA: Se detectaron inconsistencias en los datos")
    print("  Revisar preprocesado y fuentes de datos")


In [ ]:
# ANÁLISIS DE INTERSECCIONES
print("ANÁLISIS DE CONSENSO ENTRE ALGORITMOS\n")

# Outliers OCSVM vs Top ANN
intersection_outliers_ann4 = ocsvm_outliers.intersection(ann_top4)
intersection_outliers_ann10 = ocsvm_outliers.intersection(ann_top10)
intersection_outliers_consensus = ocsvm_outliers.intersection(ann_high_consensus)

print(f"Outliers OCSVM (4) vs ANN:")
print(f"  - vs Top 4 ANN: {len(intersection_outliers_ann4)} común(es) → {sorted(intersection_outliers_ann4)}")
print(f"  - vs Top 10 ANN: {len(intersection_outliers_ann10)} común(es) → {sorted(intersection_outliers_ann10)}")
print(f"  - vs Alto consenso ANN: {len(intersection_outliers_consensus)} común(es) → {sorted(intersection_outliers_consensus)}\n")

In [ ]:
# Top 10 OCSVM vs Top 10 ANN
intersection_top10 = ocsvm_top10.intersection(ann_top10)

print(f"Top 10 vs Top 10:")
print(f"  - Candidatos comunes: {len(intersection_top10)} → {sorted(intersection_top10)}")
print(f"  - Overlap rate: {len(intersection_top10)/10*100:.1f}%\n")

In [ ]:

# CARACTERIZACIÓN DE CANDIDATOS COMUNES
print("CARACTERIZACIÓN DE CANDIDATOS COMUNES\n")

all_common_candidates = intersection_outliers_ann10.union(intersection_top10)

if len(all_common_candidates) > 0:
    print(f"Candidatos con consenso entre algoritmos: {sorted(all_common_candidates)}\n")
    
    for candidate in sorted(all_common_candidates):
        # Info OCSVM
        ocsvm_info = df_unids_results[df_unids_results['number'] == candidate]
        if len(ocsvm_info) > 0:
            ocsvm_rank = ocsvm_info['Anomaly_Rank(%)'].iloc[0]
            ocsvm_score = ocsvm_info['svm_score'].iloc[0]
            is_outlier = ocsvm_info['prediction'].iloc[0] == -1
        else:
            ocsvm_rank, ocsvm_score, is_outlier = 0, 0, False
            
        # Info ANN
        ann_info = df_ann_consensus[df_ann_consensus['Source_ID'] == candidate]
        if len(ann_info) > 0:
            ann_rank = ann_info.index[0] + 1
            ann_consensus = ann_info['Mean_Prob_Consensus'].iloc[0]
            ann_std = ann_info['Std_Consensus'].iloc[0]
        else:
            ann_rank, ann_consensus, ann_std = 999, 0, 0
            
        print(f"UNID {candidate}:")
        print(f"  OCSVM: Rank {ocsvm_rank:5.1f}% | Score {ocsvm_score:8.4f} | Outlier: {'Sí' if is_outlier else 'No'}")
        print(f"  ANN:   Pos #{ann_rank:2d}     | Consenso {ann_consensus:5.3f}±{ann_std:.3f}\n")
else:
    print("NO HAY CANDIDATOS COMUNES entre outliers OCSVM y top ANN\n")


In [ ]:
# ANÁLISIS DE DIVERGENCIAS
print("ANÁLISIS DE DIVERGENCIAS\n")

ocsvm_only = ocsvm_outliers - ann_top10
ann_only = ann_top10 - ocsvm_top10

print(f"Candidatos únicos por método:")
print(f"  - Solo OCSVM outliers: {len(ocsvm_only)} → {sorted(ocsvm_only)}")
print(f"  - Solo ANN top 10: {len(ann_only)} → {sorted(ann_only)}\n")


In [ ]:
# POSICIONES CRUZADAS
print("POSICIONES CRUZADAS DE CANDIDATOS\n")

print(f"Posición de outliers OCSVM en ranking ANN:")
for candidate in sorted(ocsvm_outliers):
    ann_info = df_ann_consensus[df_ann_consensus['Source_ID'] == candidate]
    if len(ann_info) > 0:
        ann_rank = ann_info.index[0] + 1
        ann_consensus = ann_info['Mean_Prob_Consensus'].iloc[0]
        print(f"  UNID {candidate}: Posición #{ann_rank:3d} en ANN (consenso: {ann_consensus:.3f})")
    else:
        print(f"  UNID {candidate}: No encontrado en resultados ANN")

print(f"\nPosición de top 4 ANN en ranking OCSVM:")
for candidate in sorted(ann_top4):
    ocsvm_info = df_unids_results[df_unids_results['number'] == candidate]
    if len(ocsvm_info) > 0:
        ocsvm_rank = ocsvm_info['Anomaly_Rank(%)'].iloc[0]
        is_outlier = ocsvm_info['prediction'].iloc[0] == -1
        print(f"  UNID {candidate}: Anomaly Rank {ocsvm_rank:5.1f}% ({'Outlier' if is_outlier else 'Inlier'})")
    else:
        print(f"  UNID {candidate}: No encontrado en resultados OCSVM")

In [ ]:
# RESUMEN COMPARATIVO
print(f"\nRESUMEN COMPARATIVO\n")

total_candidates_combined = len(ocsvm_top10.union(ann_top10))
overlap_rate = len(intersection_top10) / 10 * 100

print(f"Métricas de consenso:")
print(f"  - Candidatos únicos combinados: {total_candidates_combined}")
print(f"  - Overlap Top 10 vs Top 10: {len(intersection_top10)}/10 ({overlap_rate:.1f}%)")
print(f"  - Complementariedad: {100-overlap_rate:.1f}%\n")

print(f"Niveles de consenso:")
if len(intersection_outliers_ann4) > 0:
    print(f"  Consenso alto: {len(intersection_outliers_ann4)} candidato(s) detectado(s) por ambos métodos")
else:
    print(f"  Consenso bajo: Sin overlap entre outliers OCSVM y top 4 ANN")

print(f"\nInterpretación:")
if overlap_rate >= 30:
    print(f"  Convergencia significativa entre algoritmos")
elif overlap_rate >= 10:
    print(f"  Convergencia moderada - métodos complementarios")
else:
    print(f"  Divergencia alta - algoritmos capturan anomalías diferentes")

# VERSIÓN 2 - ANTERIOR

In [ ]:
# Cargar resultados del consenso ANN 4F
ann_consensus = pd.read_csv('../data/results/ann/4F/consensus_analysis_ann_4f.csv')
print(f"Resultados ANN 4F cargados: {len(ann_consensus)} fuentes")

# Cargar resultados OCSVM 4F
ocsvm_results = df_unids_results.copy()  # O cargar desde archivo si está guardado
print(f"Resultados OCSVM 4F cargados: {len(ocsvm_results)} fuentes")

# Verificar mismo número de fuentes
if len(ann_consensus) != len(ocsvm_results):
    print(f" ADVERTENCIA: Diferentes números de fuentes - ANN: {len(ann_consensus)}, OCSVM: {len(ocsvm_results)}")
else:
    print(f"Mismo número de fuentes: {len(ann_consensus)}")

In [ ]:
print("Resultados (top unids) consenso ANN 4F:")
ann_consensus.head()

In [ ]:
print('Resultados (todos) OCSVM 2F:')
ocsvm_results.head()

In [ ]:
# Verificar alineamiento de IDs
ann_ids = ann_consensus['Source_ID'].values
ocsvm_ids = ocsvm_results['number'].values if 'number' in ocsvm_results.columns else ocsvm_results.index.values

ids_aligned = np.array_equal(ann_ids, ocsvm_ids)

if ids_aligned:
    print("Los IDs están alineados correctamente")
    ann_final = ann_consensus.copy()
    ocsvm_final = ocsvm_results.copy()
else:
    print("Los IDs NO están alineados - procediendo a alinear...")
    print(f"   Primeros 5 IDs ANN: {ann_ids[:5]}")
    print(f"   Primeros 5 IDs OCSVM: {ocsvm_ids[:5]}")
    
    # Alinear por ID común
    if 'number' in ocsvm_results.columns:
        ann_final = ann_consensus.sort_values('Source_ID').reset_index(drop=True)
        ocsvm_final = ocsvm_results.sort_values('number').reset_index(drop=True)
        
        # Verificar alineamiento después de ordenar
        if np.array_equal(ann_final['Source_ID'].values, ocsvm_final['number'].values):
            print("IDs alineados correctamente después de ordenar")
        else:
            print("ERROR: No se pudieron alinear los IDs - revisar datos")
            # Mostrar diferencias
            ann_set = set(ann_final['Source_ID'].values)
            ocsvm_set = set(ocsvm_final['number'].values)
            print(f"   IDs solo en ANN: {ann_set - ocsvm_set}")
            print(f"   IDs solo en OCSVM: {ocsvm_set - ann_set}")
    else:
        print("ERROR: Columna 'number' no encontrada en OCSVM results")
        ann_final = ann_consensus.copy()
        ocsvm_final = ocsvm_results.copy()

In [ ]:
# CREAR DATAFRAME COMBINADO

# Verificar columnas necesarias existen
required_ann_cols = ['Source_ID', 'E_peak', 'Beta', 'log_E_peak', 'log_Beta', 'log_Sigma_det', 'log_Beta_Rel', 'Mean_Prob_Consensus', 'Std_Prob_Consensus']
required_ocsvm_cols = ['prediction', 'Anomaly_Score', 'Anomaly_Rank(%)']

missing_ann = [col for col in required_ann_cols if col not in ann_final.columns]
missing_ocsvm = [col for col in required_ocsvm_cols if col not in ocsvm_final.columns]

if missing_ann:
    print(f"Columnas faltantes en ANN: {missing_ann}")
if missing_ocsvm:
    print(f"Columnas faltantes en OCSVM: {missing_ocsvm}")

if not missing_ann and not missing_ocsvm:
    comparison_df = pd.DataFrame({
        # IDs y características originales
        'Source_ID': ann_final['Source_ID'].values,
        'E_peak': ann_final['E_peak'].values,
        'Beta': ann_final['Beta'].values,
        'Log_E_peak': ann_final['log_E_peak'].values,
        'Log_Beta': ann_final['log_Beta'].values,
        'Log_Sigma_det': ann_final['log_Sigma_det'].values,
        'log_Beta_Rel': ann_final['log_Beta_Rel'].values,
        
        # Resultados ANN 2F
        'ANN_Prob_Mean': ann_final['Mean_Prob_Consensus'].values,
        'ANN_Prob_Std': ann_final['Std_Prob_Consensus'].values,
        
        # Resultados OCSVM 2F
        'OCSVM_Prediction': ocsvm_final['prediction'].values,  # 1=inlier, -1=outlier
        'OCSVM_Anomaly_Score': ocsvm_final['Anomaly_Score'].values,
        'OCSVM_Anomaly_Rank': ocsvm_final['Anomaly_Rank(%)'].values,
    })
    
else:
    print("No se pudo crear el DataFrame combinado debido a columnas faltantes")
    comparison_df = None

In [ ]:
# DIAGNÓSTICO DIRECTO ANN 4F vs OCSVM 4F
print("DIAGNÓSTICO ANN 4F vs OCSVM 4F")

print("-" * 60)
print(f"ANN 4F - Media: {comparison_df['ANN_Prob_Mean'].mean():.4f} | "
      f"Mediana: {comparison_df['ANN_Prob_Mean'].median():.4f} | "
      f"Rango: [{comparison_df['ANN_Prob_Mean'].min():.4f}-{comparison_df['ANN_Prob_Mean'].max():.4f}]")

print(f"OCSVM 4F - Media: {comparison_df['OCSVM_Anomaly_Rank'].mean():.1f}% | "
      f"Mediana: {comparison_df['OCSVM_Anomaly_Rank'].median():.1f}% | "
      f"Rango: [{comparison_df['OCSVM_Anomaly_Rank'].min():.1f}%-{comparison_df['OCSVM_Anomaly_Rank'].max():.1f}%]")

# IDENTIFICACIÓN DE CANDIDATOS
print("-" * 60)
print(f"\nIDENTIFICACIÓN DE CANDIDATOS:")
ann_threshold = np.percentile(comparison_df['ANN_Prob_Mean'], 95)
ocsvm_threshold = np.percentile(comparison_df['OCSVM_Anomaly_Rank'], 95)

top_ann = comparison_df[comparison_df['ANN_Prob_Mean'] >= ann_threshold].sort_values('ANN_Prob_Mean', ascending=False)
ocsvm_outliers = comparison_df[comparison_df['OCSVM_Prediction'] == -1].sort_values('OCSVM_Anomaly_Rank', ascending=False)
top_ocsvm = comparison_df[comparison_df['OCSVM_Anomaly_Rank'] >= ocsvm_threshold].sort_values('OCSVM_Anomaly_Rank', ascending=False)

print(f"Top ANN (P95={ann_threshold:.4f}): {len(top_ann)} fuentes")
print(f"OCSVM Outliers: {len(ocsvm_outliers)} fuentes")
print(f"Top OCSVM (P95={ocsvm_threshold:.1f}%): {len(top_ocsvm)} fuentes")

# SOLAPAMIENTOS
print(f"\nSOLAPAMIENTOS:")
overlap_restrictivo = set(top_ann['Source_ID']) & set(ocsvm_outliers['Source_ID'])
overlap_amplio = set(top_ann['Source_ID']) & set(top_ocsvm['Source_ID'])

print(f"Top ANN ∩ OCSVM Outliers: {len(overlap_restrictivo)} fuentes")
if overlap_restrictivo:
    print(f"  IDs: {sorted(list(overlap_restrictivo))}")

print(f"Top ANN ∩ Top OCSVM: {len(overlap_amplio)} fuentes")
if overlap_amplio:
    print(f"  IDs: {sorted(list(overlap_amplio))}")

# TOP 10 COMPARACIÓN DIRECTA
print(f"\nTOP 10 ANN vs OCSVM:")
print("-" * 60)
for i, (_, row) in enumerate(top_ann.head(10).iterrows()):
    status = "Outlier" if row['OCSVM_Prediction'] == -1 else "Normal"
    print(f"{i+1:2d}. ID {int(row['Source_ID']):4d}: ANN={row['ANN_Prob_Mean']:.4f} | OCSVM={row['OCSVM_Anomaly_Rank']:5.1f}% {status}")

print(f"\nOUTLIERS OCSVM vs ANN:")
print("-" * 60)
if len(ocsvm_outliers) > 0:
    for i, (_, row) in enumerate(ocsvm_outliers.iterrows()):
        print(f"{i+1:2d}. ID {int(row['Source_ID']):4d}: OCSVM={row['OCSVM_Anomaly_Rank']:5.1f}% | ANN={row['ANN_Prob_Mean']:.4f}")
else:
    print("No hay outliers detectados por OCSVM")

# ANÁLISIS DE CONCORDANCIA
print(f"\nANÁLISIS DE CONCORDANCIA:")
# Correlación entre métricas
correlation = np.corrcoef(comparison_df['ANN_Prob_Mean'], comparison_df['OCSVM_Anomaly_Rank'])[0, 1]
print(f"Correlación ANN-OCSVM: {correlation:.4f}")

# Concordancia en clasificación binaria
ann_high = comparison_df['ANN_Prob_Mean'] >= ann_threshold
ocsvm_high = comparison_df['OCSVM_Anomaly_Rank'] >= ocsvm_threshold

concordancia = np.mean(ann_high == ocsvm_high) * 100
print(f"Concordancia en top 5%: {concordancia:.1f}%")

# Casos discrepantes
discrepantes_ann_high_ocsvm_low = comparison_df[(comparison_df['ANN_Prob_Mean'] >= ann_threshold) & 
                                               (comparison_df['OCSVM_Anomaly_Rank'] < 50)]
discrepantes_ocsvm_high_ann_low = comparison_df[(comparison_df['OCSVM_Anomaly_Rank'] >= ocsvm_threshold) & 
                                               (comparison_df['ANN_Prob_Mean'] < 0.5)]

print(f"Discrepantes (ANN alta, OCSVM baja): {len(discrepantes_ann_high_ocsvm_low)}")
print(f"Discrepantes (OCSVM alta, ANN baja): {len(discrepantes_ocsvm_high_ann_low)}")

print("-" * 60)
# RESUMEN EJECUTIVO
print(f"\nRESUMEN EJECUTIVO:")
if len(overlap_restrictivo) > 0:
    print(f"Consenso encontrado: {len(overlap_restrictivo)} fuentes identificadas por ambos métodos")
else:
    print("Sin consenso directo entre métodos")

if abs(correlation) > 0.5:
    print(f"Correlación moderada-alta: {correlation:.3f}")
else:
    print(f"Correlación baja: {correlation:.3f}")

if concordancia > 70:
    print(f"Buena concordancia en top candidatos: {concordancia:.1f}%")
else:
    print(f"Baja concordancia en top candidatos: {concordancia:.1f}%")